<h1 style="color : red; text-align:center; font-size:40px;">Sentiment Analysis with Bert</h1>


In [1]:
#!pip install requests beautifulsoup4

# 1.Import dependencies

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re 

# 2.Instantiate Model

In [3]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')


C:\Users\USER\anaconda3\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


This line initializes a tokenizer object for the BERT-based model called "nlptown/bert-base-multilingual-uncased-sentiment". The tokenizer is responsible for converting text inputs into tokenized format suitable for the model.

In [4]:
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

This line initializes a model object for the same BERT-based model. The model is specifically fine-tuned for sequence classification tasks. It's capable of analyzing sequences of text and making predictions about their sentiment.

In [5]:
# Check if the model is loaded
if model is not None:
    print("Model has been successfully loaded.")
else:
    print("Model loading failed.")

Model has been successfully loaded.


# 2.Encode and Calculate Sentiment

In [6]:
tokens = tokenizer.encode('I hated this , absolutely the worst',return_tensors='pt')


In [9]:
tokens[0]

tensor([  101,   151, 39487, 10163, 10372,   117, 35925, 10563, 10103, 43060,
          102])

In [10]:
tokenizer.decode(tokens[0])

'[CLS] i hated this, absolutely the worst [SEP]'

In [11]:
result= model(tokens)

In [12]:
result

SequenceClassifierOutput(loss=None, logits=tensor([[ 4.8750,  1.7880, -0.8356, -3.0027, -2.0727]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [13]:
# the values represent the prob of that class being
#the sentimennt ,, the position 0 is the highest
#the position 0 is the sentiment (which is hatered)

In [14]:
result.logits

tensor([[ 4.8750,  1.7880, -0.8356, -3.0027, -2.0727]],
       grad_fn=<AddmmBackward0>)

In [15]:
int(torch.argmax(result.logits))+1


1

In [16]:
#if we change the sentence the value change between 1 and 5

# 4.Collect Reviews

In [17]:
r = requests.get('https://www.yelp.com/biz/mejico-sydney-2')
soup= BeautifulSoup(r.text, 'html.parser')
regex= re.compile('.*comment.*')
results= soup.find_all('p',{'class':regex})
reviews = [result.text for result in results]

### each of the comments are actually stored within a class that starts with comment
(right click on the site , inspecter(last option ) to find out 


In [18]:
r

<Response [200]>

In [ ]:
r.text

In [ ]:
soup

In [ ]:
soup.find

In [ ]:
results

In [22]:
# results give us all the reviews in that site

In [23]:
results[0].text

'The food was decent not great..  We had the guacamole which was bland and came with some type of plantain chips.. The chicken and steak tacos were good.. But the service was poor. We had a waitress with an attitude. She seemed upset whenever we asked for anything.  She would walk by and just stick up her hand and say " just wait ".  She spilled the ingredients to make the guacamole all over the table but never apologized. The waitress didn\'t come by at all, not even once to check on us.. I will not go back. Suggest you find a Mexican restaurant that really wants your business..'

# 5.Load Reviews into DataFrame and Score

In [24]:
import numpy as np 
import pandas as pd 

In [25]:
df = pd.DataFrame(np.array(reviews), columns=['review'])


In [26]:
df.iloc[0]

review    The food was decent not great..  We had the gu...
Name: 0, dtype: object

In [27]:
df['review'].iloc[0]

'The food was decent not great..  We had the guacamole which was bland and came with some type of plantain chips.. The chicken and steak tacos were good.. But the service was poor. We had a waitress with an attitude. She seemed upset whenever we asked for anything.  She would walk by and just stick up her hand and say " just wait ".  She spilled the ingredients to make the guacamole all over the table but never apologized. The waitress didn\'t come by at all, not even once to check on us.. I will not go back. Suggest you find a Mexican restaurant that really wants your business..'

In [28]:
def sentiment_score(review):
    tokens= tokenizer.encode(review,return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits)+1)

In [29]:
sentiment_score(df['review'].iloc[1])

2

In [30]:
df['sentiment']= df['review'].apply(lambda x: sentiment_score(x[:512]))

In [31]:
df['review']

0    The food was decent not great..  We had the gu...
1    Food was okay, guacamole was below average. Se...
2    The food and service here was really good.  It...
3    Visiting from Texas and decided to give this r...
4    Don't come here expecting legit Mexican food b...
5    Out of all the restaurants that I tried in Syd...
6    Great atmosphere, attentive service, solid mar...
7    We came here on a Thursday night @ 5pm and by ...
8    The food is fresh and tasty.  The scallop cevi...
9    Have been here twice and have absolutely loved...
Name: review, dtype: object

In [32]:
df

,review,sentiment
0,The food was decent not great.. We had the gu...,2
1,"Food was okay, guacamole was below average. Se...",2
2,The food and service here was really good. It...,5
3,Visiting from Texas and decided to give this r...,5
4,Don't come here expecting legit Mexican food b...,3
5,Out of all the restaurants that I tried in Syd...,5
6,"Great atmosphere, attentive service, solid mar...",3
7,We came here on a Thursday night @ 5pm and by ...,4
8,The food is fresh and tasty. The scallop cevi...,4
9,Have been here twice and have absolutely loved...,5


In [33]:
df['review'].iloc[3]

'Visiting from Texas and decided to give this restaurant a try.  We were pleasantly surprised.  While the margaritas are more like martinis, the food was excellent.  More like a tapas, Mexican fusion.  Great way to try different plates.The real treat was Chelsea, our waitress.  Took the time to explain the menu and offer suggestions.  Always smiling and very pleasant.  Best service we have had in Sydney!'